In [ ]:

# Instalación necesaria
!pip install yfinance --quiet

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from scipy.optimize import minimize

# === Entrada del usuario ===
tickers_input = input("Introduce los símbolos bursátiles separados por comas: ")
tickers = [ticker.strip().upper() for ticker in tickers_input.split(',')]

# === Descarga de datos históricos ===
# Recomendamos una ventana más amplia para mayor robustez
data = yf.download(tickers, start="2020-01-01", end="2025-06-01")['Close']

# Cálculo de retornos logarítmicos
returns = np.log(data / data.shift(1)).dropna()

# Eliminar activos con demasiados datos faltantes
returns = returns.dropna(axis=1, thresh=int(0.95 * len(returns)))
tickers = returns.columns.tolist()

mean_returns = returns.mean()
cov_matrix = returns.cov()
num_assets = len(tickers)

# === Parámetros de optimización ===
risk_free_rate = np.log(1.01) / 252  # tasa libre de riesgo diaria logarítmica
init_guess = num_assets * [1. / num_assets]
bounds = tuple((0, 1) for _ in range(num_assets))
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

# === Funciones auxiliares ===
def portfolio_performance(weights, mean_returns, cov_matrix):
    ret = np.dot(weights, mean_returns)
    vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return ret, vol

def negative_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate):
    ret, vol = portfolio_performance(weights, mean_returns, cov_matrix)
    return -(ret - risk_free_rate) / vol

# === Optimización del ratio de Sharpe ===
sharpe_result = minimize(negative_sharpe_ratio, init_guess, args=(mean_returns, cov_matrix, risk_free_rate),
                         method='SLSQP', bounds=bounds, constraints=constraints)

opt_weights = sharpe_result.x
opt_return, opt_risk = portfolio_performance(opt_weights, mean_returns, cov_matrix)
sharpe_ratio = (opt_return - risk_free_rate) / opt_risk

# === Resultados ===
print("Ponderaciones óptimas:")
for ticker, weight in zip(tickers, opt_weights):
    print(f"{ticker}: {weight:.2%}")
print(f"Rendimiento esperado: {opt_return:.2%}")
print(f"Riesgo (volatilidad): {opt_risk:.2%}")
print(f"Ratio de Sharpe: {sharpe_ratio:.2f}")

# === Cálculo de la Frontera Eficiente ===
target_returns = np.linspace(0.0, 0.01, 100)
efficient_portfolios = []

for r in target_returns:
    constraints_eff = (
        {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
        {'type': 'eq', 'fun': lambda x: np.dot(x, mean_returns) - r}
    )
    result = minimize(lambda w: portfolio_performance(w, mean_returns, cov_matrix)[1],
                      init_guess, method='SLSQP', bounds=bounds, constraints=constraints_eff)
    if result.success:
        efficient_portfolios.append(result)

eff_returns = [portfolio_performance(p.x, mean_returns, cov_matrix)[0] for p in efficient_portfolios]
eff_risks = [portfolio_performance(p.x, mean_returns, cov_matrix)[1] for p in efficient_portfolios]

# === Gráfico ===
plt.figure(figsize=(10, 6))
plt.plot(eff_risks, eff_returns, label='Frontera Eficiente')
plt.plot([0, opt_risk], [risk_free_rate, opt_return], 'r--', label='Línea CAL')
plt.scatter(opt_risk, opt_return, c='red', marker='*', s=100, label='Cartera Óptima Sharpe')
plt.xlabel('Riesgo (Volatilidad)')
plt.ylabel('Rendimiento Esperado')
plt.title('Frontera Eficiente y Cartera Óptima')
plt.legend()
plt.grid(True)
plt.show()
